In [5]:
from pringles.simulator import Simulator
mySimulator = Simulator(cdpp_bin_path='bin/', user_models_dir='src/')

In [6]:
for atomic in mySimulator.atomic_registry.discovered_atomics:
    if atomic.__name__ == "Server":
        Server = atomic
    elif atomic.__name__ == "AutoScaler":
        AutoScaler = atomic
    elif atomic.__name__ == "ServerQueue":
        ServerQueue = atomic
    elif atomic.__name__ == "Dispatcher":
        Dispatcher = atomic

print(Server)
print(Dispatcher)

<class 'pringles.models.models.Server'>
<class 'pringles.models.models.Dispatcher'>


In [7]:
from pringles.models import Coupled
number_of_servers = 3
a_dispatcher = Dispatcher("a_dispatcher", 
                          numberOfServers=number_of_servers, 
                          server0="free",
                          server1="free",
                          server2="free")

servers = {}
for i in range(number_of_servers):
    servers[i] = Server("server" + str(i), initialStatus="free", mean=10.0, setupTime="0:0:10:0")
    
print("--Dispatcher ports")
print("Inport names: ", [port.name for port in a_dispatcher.inports])
print("Outport names: ", [port.name for port in a_dispatcher.outports])

print("--Server ports--")
print("Inport names: ", [port.name for port in servers[0].inports])
print("Outport names: ", [port.name for port in servers[0].outports])

--Dispatcher ports
Inport names:  ['newJob', 'jobDone', 'serverStackInfo']
Outport names:  ['requestJob', 'server0', 'server1', 'server2']
--Server ports--
Inport names:  ['job', 'powerSignal']
Outport names:  ['done', 'ready']


In [8]:
subcomponents = [a_dispatcher]
for i in range(number_of_servers):
    subcomponents.append(servers[i])
    
top_model = (Coupled(name='top', subcomponents=subcomponents)
             .add_inport("newJob")
             .add_inport("jobDone")             
             .add_inport("serverStackInfo")
             .add_outport("requestJob")
#              .add_outport("server0")
#              .add_outport("server1")
#              .add_outport("server2")
             .add_coupling('newJob', a_dispatcher.get_port("newJob"))
             .add_coupling('jobDone', a_dispatcher.get_port("jobDone"))
             .add_coupling('serverStackInfo', a_dispatcher.get_port("serverStackInfo"))
             
             .add_coupling(a_dispatcher.get_port('requestJob'), "requestJob")
             .add_coupling(a_dispatcher.get_port('server0'), servers[0].get_port('job'))
             .add_coupling(a_dispatcher.get_port('server1'), servers[1].get_port('job'))
             .add_coupling(a_dispatcher.get_port('server2'), servers[2].get_port('job'))
             
             .add_coupling(servers[0].get_port('done'), a_dispatcher.get_port('jobDone'))
             .add_coupling(servers[1].get_port('done'), a_dispatcher.get_port('jobDone'))
             .add_coupling(servers[2].get_port('done'), a_dispatcher.get_port('jobDone'))
             
            )
top_model

In [9]:
from pringles.simulator import Simulation, Event
from pringles.utils import VirtualTime

In [10]:
# sim_events = [
#     Event(VirtualTime(0,0,15,0,0), top_model.get_port('newJob'), float(1)),
#     Event(VirtualTime(0,0,20,0,0), top_model.get_port('jobDone'), float(0)),
#     Event(VirtualTime(0,0,25,0,0), top_model.get_port('newJob'), float(1)),
#     Event(VirtualTime(0,0,30,0,0), top_model.get_port('jobDone'), float(1))
# ]

# events 09
# sim_events = [
#     Event(VirtualTime(0,0,15,0,0), top_model.get_port('serverStackInfo'), [float(0), float(1)])
# ]

# events 15

# Empiezan los 3 servidores prendidos. Y se avisa que el 2 está apagado (habría que mandarle tmb que se apague)
# Llega un Job a los 25 y el dispatcher lo debería mandar al server 0 porque es el de menor ID
# Llega que se prendió de nuevo el server 2
# Llega otro Job a los 38 y se los manda de nuevo al server 0 (en la simulación el server 0 termina el job anterior
    # en el segundo 36)
# Llega en 40 que se apagó el 0, pero el lo tenía en busy, espera que se apague y lo pasa a off
# Llega en 42 un Job y como el 0 está en apagado/busy, se lo manda al server 1
# Llega en 43 que se apague el 1, espera que se termine el job y lo apaga
# Llega en 44 un Job y se lo manda al server 2, el único prendido
sim_events = [
    Event(VirtualTime(0,0,20,0,0), top_model.get_port('serverStackInfo'), [float(2),float(0)]),
    Event(VirtualTime(0,0,25,0,0), top_model.get_port('newJob'), float(1)),
    Event(VirtualTime(0,0,35,0,0), top_model.get_port('serverStackInfo'), [float(2),float(1)]),
    Event(VirtualTime(0,0,38,0,0), top_model.get_port('newJob'), float(1)),
    Event(VirtualTime(0,0,40,0,0), top_model.get_port('serverStackInfo'), [float(0),float(0)]),
    Event(VirtualTime(0,0,42,0,0), top_model.get_port('newJob'), float(1)),
    Event(VirtualTime(0,0,43,0,0), top_model.get_port('serverStackInfo'), [float(1),float(0)]),
    Event(VirtualTime(0,0,44,0,0), top_model.get_port('newJob'), float(1)),
]

a_simulation = Simulation(top_model = top_model, 
                          duration = VirtualTime.of_minutes(50), 
                          events=sim_events,
                          working_dir='sim_results/'
                         )

results = mySimulator.run_simulation(a_simulation)

In [11]:
print(results.get_process_output())

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from sim_results/2019-10-09-195916-cbad3a29769040aba2832b90d08b028f/top_model
Loading events from sim_results/2019-10-09-195916-cbad3a29769040aba2832b90d08b028f/events
Running parallel simulation. Reading models partition from 
Model partition details output to: /dev/null*
Mess

In [31]:
print(results.get_process_error_output())

At 00:00:47:954:0.305649


In [38]:
display(results.output_df.head(100))

,time,port,value
0,00:00:00:000,requestjob,1.0
1,00:00:25:000,requestjob,1.0
2,00:00:38:000,requestjob,1.0
3,00:00:42:000,requestjob,1.0
4,00:00:44:330,requestjob,1.0
5,00:01:00:239,requestjob,1.0


In [33]:
print(results.logs_dfs.keys(),'\n\n')
display(results.logs_dfs['ParallelRoot'].head())

dict_keys(['server2', 'server1', 'server0', 'a_dispatcher', 'top', 'ParallelRoot']) 




,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,Y,00:00:00:000,top(05),requestjob,1.0,ParallelRoot(00)
1,0,L,Y,00:00:25:000,top(05),requestjob,1.0,ParallelRoot(00)
2,0,L,Y,00:00:38:000,top(05),requestjob,1.0,ParallelRoot(00)
3,0,L,Y,00:00:42:000,top(05),requestjob,1.0,ParallelRoot(00)
4,0,L,Y,00:00:44:330,top(05),requestjob,1.0,ParallelRoot(00)


Como podemos observar, los mensajes que siguen abajo, son los mismos que en el notebook sin la Queue.

In [34]:
# Veamos que le llegan al server 0 los primneros 2 jobs, hasta que lo apagan en segundo 40
display(results.logs_dfs['server0'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:25:000,top(05),job,0.0,server0(02)
1,0,L,X,00:00:38:000,top(05),job,1.0,server0(02)


In [35]:
# Veamos que el siguiente job le llega al server 1 en el segundo 42, y luego lo apagan en el segundo 43
# Y se espera a que termine para apagarlo
display(results.logs_dfs['server1'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:42:000,top(05),job,2.0,server1(03)


In [36]:
# Por último cuando el 0 y el 1 están apagados, cuando llega el último job en el segundo 44, se lo envía al server 2
# que lo termina (ver logs dispatcher)
display(results.logs_dfs['server2'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:44:000,top(05),job,3.0,server2(04)


In [37]:
display(results.logs_dfs['a_dispatcher'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:20:000,top(05),serverstackinfo,"(2.0, 0.0)",a_dispatcher(01)
1,0,L,X,00:00:25:000,top(05),newjob,1,a_dispatcher(01)
2,0,L,X,00:00:31:116,top(05),jobdone,0,a_dispatcher(01)
3,0,L,X,00:00:35:000,top(05),serverstackinfo,"(2.0, 1.0)",a_dispatcher(01)
4,0,L,X,00:00:38:000,top(05),newjob,1,a_dispatcher(01)
5,0,L,X,00:00:40:000,top(05),serverstackinfo,"(0.0, 0.0)",a_dispatcher(01)
6,0,L,X,00:00:42:000,top(05),newjob,1,a_dispatcher(01)
7,0,L,X,00:00:42:777,top(05),jobdone,2,a_dispatcher(01)
8,0,L,X,00:00:43:000,top(05),serverstackinfo,"(1.0, 0.0)",a_dispatcher(01)
9,0,L,X,00:00:44:000,top(05),newjob,1,a_dispatcher(01)


# Ahora vamos a probar con una simulación un poco más grande

In [32]:
sim_events = [
    Event(VirtualTime(0,0,20,0,0), top_model.get_port('serverStackInfo'), [float(2),float(0)]),
    Event(VirtualTime(0,0,25,0,0), top_model.get_port('newJob'), float(1)),
    Event(VirtualTime(0,0,35,0,0), top_model.get_port('serverStackInfo'), [float(2),float(1)]),
    Event(VirtualTime(0,0,38,0,0), top_model.get_port('newJob'), float(1)),
    Event(VirtualTime(0,0,40,0,0), top_model.get_port('serverStackInfo'), [float(0),float(0)]),
    Event(VirtualTime(0,0,42,0,0), top_model.get_port('newJob'), float(1)),
    Event(VirtualTime(0,0,43,0,0), top_model.get_port('serverStackInfo'), [float(1),float(0)]),
    Event(VirtualTime(0,0,44,0,0), top_model.get_port('newJob'), float(1)),
    Event(VirtualTime(0,0,60,0,0), top_model.get_port('newJob'), float(1)),
#     Event(VirtualTime(0,0,60,0,0), top_model.get_port('newJob'), float(1)),
#     Event(VirtualTime(0,0,60,0,0), top_model.get_port('newJob'), float(1)),
#     Event(VirtualTime(0,0,60,0,0), top_model.get_port('newJob'), float(1)),
#     Event(VirtualTime(0,1,2,0,0), top_model.get_port('newJob'), float(1)),
#     Event(VirtualTime(0,1,2,0,0), top_model.get_port('newJob'), float(1)),
]

a_simulation = Simulation(top_model = top_model, 
                          duration = VirtualTime.of_minutes(50), 
                          events=sim_events,
                          working_dir='sim_results/'
                         )

results = mySimulator.run_simulation(a_simulation)

In [42]:
display(results.logs_dfs['server0'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:25:000,top(05),job,0.0,server0(02)
1,0,L,X,00:00:38:000,top(05),job,1.0,server0(02)


In [36]:
display(results.logs_dfs['server1'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:42:000,top(05),job,2.0,server1(03)


In [37]:
display(results.logs_dfs['server2'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:44:000,top(05),job,3.0,server2(04)
1,0,L,X,00:01:00:000,top(05),job,4.0,server2(04)


In [29]:
display(results.logs_dfs['a_dispatcher'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:20:000,top(05),serverstackinfo,"(2.0, 0.0)",a_dispatcher(01)
1,0,L,X,00:00:25:000,top(05),newjob,1,a_dispatcher(01)
2,0,L,X,00:00:31:116,top(05),jobdone,0,a_dispatcher(01)
3,0,L,X,00:00:35:000,top(05),serverstackinfo,"(2.0, 1.0)",a_dispatcher(01)
4,0,L,X,00:00:38:000,top(05),newjob,1,a_dispatcher(01)
5,0,L,X,00:00:40:000,top(05),serverstackinfo,"(0.0, 0.0)",a_dispatcher(01)
6,0,L,X,00:00:42:000,top(05),newjob,1,a_dispatcher(01)
7,0,L,X,00:00:42:777,top(05),jobdone,2,a_dispatcher(01)
8,0,L,X,00:00:43:000,top(05),serverstackinfo,"(1.0, 0.0)",a_dispatcher(01)
9,0,L,X,00:00:44:000,top(05),newjob,1,a_dispatcher(01)


In [34]:
print(results.get_process_output())

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from sim_results/2019-10-09-202903-a13aebb677f94d7f997dd078d3262b81/top_model
Loading events from sim_results/2019-10-09-202903-a13aebb677f94d7f997dd078d3262b81/events
Running parallel simulation. Reading models partition from 
Model partition details output to: /dev/null*
Mess

In [33]:
print(results.get_process_error_output())

At 00:01:00:239:0.408016
